<a href="https://colab.research.google.com/github/pachterlab/JBMMCKP_2023/blob/main/cdx_liver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Enable R on Colab above with `Runtime > Change Runtime Type`

In [ ]:
system("git clone https://github.com/pachterlab/JBMMCKP_2023.git")
setwd("JBMMCKP_2023")

These dependencies need special attention on Colab

In [ ]:
# Installation of  `magick` is req'd for some BioC packages,
# but installation is halted on Colab
system('add-apt-repository -y ppa:cran/imagemagick')
system('apt-get update')
system("apt-get install libmagick++-dev")
system("sudo apt-get install libgmp-dev")

install.packages("magick")
install.packages("fastcluster")

Install Bioconductor packages

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.19")

# Install Bioconductor packages
BiocManager::install("SpatialExperiment")
BiocManager::install("bluster")
BiocManager::install("scater")
BiocManager::install("mbkmeans")

In [ ]:
devtools::install_github("pachterlab/concordexR")

Load  `concordexR` and some extra libraries for plotting and utility functions.

In [ ]:
library(tidyr)
library(dplyr)

library(ggplot2)
library(scattermore)
library(fs)
library(readr)


library(concordexR)

data_dir <- fs::path_wd("data")

# Load Data

In [ ]:
meta <- read_csv(path(data_dir,"liver/cell_meta.csv"))

# concordex Results

In [ ]:
# defaults to 30 nn
set.seed(5296)

coords <- meta[,c("center_x","center_y")]

cdx <- calculateConcordex(
    x=coords,
    labels=as.character(meta$leiden),
    BLUSPARAM=bluster::MbkmeansParam(4,50))

cdx_pred <- attr(cdx, "shrs")
pl_data <- dplyr::mutate(coords, shr=cdx_pred)

# Plot SHRs

In [ ]:
p <- ggplot(pl_data, aes(center_x, center_y, color=shr)) +
    geom_scattermore() +
    theme_minimal()

p

In [ ]:
mask <- between(meta$fov, 500, 999) # to match np.arange(500,1000)


cdx_small <- calculateConcordex(
    x=coords[mask,],
    labels=as.character(meta$leiden)[mask],
    BLUSPARAM=bluster::MbkmeansParam(4,50))


cdx_pred <- attr(cdx, "shrs")
pl_data <- dplyr::mutate(coords, shr=cdx_pred)


# Session Info

In [ ]:
sessionInfo()